In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2 as cv
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [ ]:
import tensorflow as tf

In [ ]:
DataDir = '/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b'
Categories = ['aluminium_foil', 'brown_bread', 'corduroy', 'cork', 'cotton', 'cracker', 'lettuce_leaf', 'linen', 'white_bread', 'wood','wool']
Samples = ['sample_a']
images = []
y = np.array([])

In [ ]:
for category in Categories:
  for Sample in Samples:
    path = os.path.join(DataDir,category)
    path = os.path.join(path,Sample)
    class_num = Categories.index(category)
    print(path)
    for imgname in os.listdir(path):
        img = cv.imread(os.path.join(path,imgname))
        img = cv.resize(img, (300, 300))
        images.append(img)
        y=np.append(y,[class_num])
      
        
  
x = np.array(images)
print(x.shape)

/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/aluminium_foil/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/brown_bread/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/corduroy/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/cork/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/cotton/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/cracker/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/lettuce_leaf/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/linen/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/white_bread/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/wood/sample_a
/content/drive/MyDrive/PS 1/Datasets/KTH-TIPS-2B/KTH-TIPS2-b/wool/sample_a
(1188, 300, 300, 3)


In [ ]:
y = np.reshape(y,(-1,1))
print(y.shape)
print(y)

(1188, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 ...
 [10.]
 [10.]
 [10.]]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
val_ds_size = tf.data.experimental.cardinality(val_ds).numpy()

print('Train size:', train_ds_size)
print('Test size:', test_ds_size)
print('Val size:', val_ds_size)

Train size: 712
Test size: 238
Val size: 238


In [ ]:
def process_image(image,label):
    image=tf.image.per_image_standardization(image)
    image=tf.image.resize(image,(224,224))
    
    return image,label

In [ ]:
train_ds = (train_ds
          .map(process_image)
          .shuffle(buffer_size=train_ds_size)
          .batch(batch_size=30,drop_remainder=True)
         )

test_ds = (test_ds
           .map(process_image)
          .shuffle(buffer_size=test_ds_size)
          .batch(batch_size=30,drop_remainder=True)
         )

val_ds = (val_ds
          .map(process_image)
          .shuffle(buffer_size=val_ds_size)
          .batch(batch_size=30,drop_remainder=True)
         )

In [ ]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
#Base Model
base_model = ResNet50(input_shape=(224, 224, 3))

In [ ]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 11,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

In [ ]:
model = Model(inputs=base_model.input, outputs=headModel)

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.optimizers.SGD(lr=0.003),
    metrics=['accuracy']    
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(train_ds,
          epochs=200,
          validation_data=val_ds,
          validation_freq=1)

Epoch 1/200
23/23 [==============================] - 69s 934ms/step - loss: 2.7883 - accuracy: 0.1713 - val_loss: 2.4012 - val_accuracy: 0.1143
Epoch 2/200
23/23 [==============================] - 17s 724ms/step - loss: 1.9837 - accuracy: 0.2977 - val_loss: 2.4822 - val_accuracy: 0.1000
Epoch 3/200
23/23 [==============================] - 18s 725ms/step - loss: 1.7329 - accuracy: 0.3460 - val_loss: 2.7226 - val_accuracy: 0.0905
Epoch 4/200
23/23 [==============================] - 17s 723ms/step - loss: 1.5710 - accuracy: 0.4196 - val_loss: 2.8306 - val_accuracy: 0.1048
Epoch 5/200
23/23 [==============================] - 17s 722ms/step - loss: 1.1301 - accuracy: 0.6028 - val_loss: 2.9574 - val_accuracy: 0.0857
Epoch 6/200
23/23 [==============================] - 18s 722ms/step - loss: 1.0756 - accuracy: 0.5598 - val_loss: 3.1022 - val_accuracy: 0.0905
Epoch 7/200
23/23 [==============================] - 17s 723ms/step - loss: 0.8215 - accuracy: 0.6967 - val_loss: 3.1549 - val_accuracy:

In [ ]:
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label = 'Testing accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()
%matplotlib inline